In [1]:
%load_ext watermark

In [2]:
%watermark

Last updated: 2023-03-07T10:52:29.728934-05:00

Python implementation: CPython
Python version       : 3.7.11
IPython version      : 7.10.1

Compiler    : GCC 7.5.0
OS          : Linux
Release     : 4.15.0-161-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 40
Architecture: 64bit



In [3]:
import numpy as np
import pandas as pd
import gc
import time

from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from tqdm import tqdm

import warnings
from time import time
import xgboost as xgb
xgb.__version__

'1.6.2'

In [4]:
%watermark --iversions

xgboost: 1.6.2
numpy  : 1.19.2
pandas : 1.3.5



In [5]:
train = pd.read_csv('../input/train.csv.gz')

In [6]:
target = train.label.values

In [7]:
train.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
good_features = train.columns[1:]
good_features

Index(['1x1', '1x2', '1x3', '1x4', '1x5', '1x6', '1x7', '1x8', '1x9', '1x10',
       ...
       '28x19', '28x20', '28x21', '28x22', '28x23', '28x24', '28x25', '28x26',
       '28x27', '28x28'],
      dtype='object', length=784)

In [24]:
%%time
kf = StratifiedKFold(5, shuffle=True, random_state=1974)

for i, (train_index, test_index) in enumerate(kf.split(train,target)):

        print(i)
        xgtrain, xgval = train[good_features].loc[train_index].values, train[good_features].loc[test_index].values
        y_train, y_val = target[train_index], target[test_index]
        
        trains = []
        for ii in range(5):
            trains.append(xgtrain)
            
        xgtrain_5 = np.vstack(trains)
        
        xgtrain_5 = pd.DataFrame(xgtrain_5, columns=good_features)
        xgval = pd.DataFrame(xgval, columns=good_features)
        
        y_train_5 = []
        for ii in range(5):
            y_train_5.append(y_train)
            
        y_train_5 = np.hstack(y_train_5)
        
        xgtrain_5['target'] = y_train_5
        xgval['target'] = y_val
        
        xgtrain_5 = xgtrain_5.sample(frac=1).reset_index(drop=True)


        
        xgtrain_5.to_csv(f'../input/xgtrain_5x_fold_{i}.csv.gz', index=False, compression='gzip')
        #xgval.to_csv(f'../input/xgval_fold_{i}.csv.gz', index=False, compression='gzip')

0
1
2
3
4
CPU times: user 7min 27s, sys: 5.01 s, total: 7min 32s
Wall time: 7min 32s


In [25]:
xgtrain_5.shape

(240000, 785)

In [26]:
xgval.shape

(12000, 785)